In [27]:

import grpc
import acquisition_pb2
import acquisition_pb2_grpc
import io
from PIL import Image
import json
import time
import os


In [28]:

# Function to convert PIL Image to bytes
def image_to_bytes(img: Image.Image, format='PNG'):
    """Converts a PIL Image to a bytes object."""
    buf = io.BytesIO()
    img.save(buf, format=format)
    return buf.getvalue()

# Function to create a dummy image for testing
def create_dummy_image(width=100, height=100, color=(255, 0, 0)):
    """Creates a simple red square PIL image."""
    img = Image.new('RGB', (width, height), color)
    return img


In [29]:

def run():
    # Replace with your server's address and port
    # Assuming the server is running on localhost:8061 (default from your server code)
    channel = grpc.insecure_channel('localhost:8061', options=[
        ('grpc.max_send_message_length', 512 * 1024 * 1024),
        ('grpc.max_receive_message_length', 512 * 1024 * 1024)
    ])
    stub = acquisition_pb2_grpc.AcquisitionServiceStub(channel)

    print("--- Testing Acquire Method ---")
    try:
        # For 'acquire' method, the server expects something to be in its 'data_acq_queue'.
        # In a real scenario, another part of your system or the Gradio UI would put data there.
        # For this test, we'll simulate the server having data ready.
        # Since the server's 'acquire' method simply gets from 'data_acq_queue',
        # we can only test the 'No data available' scenario unless we modify the server
        # to have initial data or interact with its Gradio interface first.

        # Let's assume the server has processed an image from its Gradio input
        # and it's available in its internal queue.
        # To make this test truly work with your server, you'd either:
        # 1. Manually upload an image to the Gradio UI on the server.
        # 2. Modify the server to initially populate `data_acq_queue` with dummy data for testing.
        # 3. (Less common for gRPC) Have a separate thread in the client that puts data into the server's queue.

        print("Attempting to acquire data...")
        acquire_request = acquisition_pb2.AcquireRequest()
        acquire_response = stub.acquire(acquire_request)

        if acquire_response.label and acquire_response.image:
            print(f"Acquire successful!")
            print(f"Received label: {acquire_response.label}")

            # Save the acquired image
            try:
                acquired_image_path = "acquired_image_from_grpc.png"
                with open(acquired_image_path, "wb") as f:
                    f.write(acquire_response.image)
                print(f"Acquired image saved to {acquired_image_path}")
                # Optional: Open the image to verify
                # Image.open(io.BytesIO(acquire_response.image)).show()
            except Exception as e:
                print(f"Error saving acquired image: {e}")
        else:
            print("Acquire returned an empty response. This might mean no data was available on the server.")

    except grpc.RpcError as e:
        if e.code() == grpc.StatusCode.NOT_FOUND:
            print(f"Acquire failed: {e.details}")
        else:
            print(f"Acquire failed with unexpected error: {e}")

    print("\n--- Testing Display Method ---")
    try:
        # Create a dummy image to send for display
        #dummy_image = create_dummy_image(width=200, height=150, color=(0, 255, 0)) # Green image
        #dummy_image_bytes = image_to_bytes(dummy_image, format='JPEG') # Send as JPEG for variety
        dummy_image_bytes=acquire_response.image

        # Create a dummy label (JSON string)
        dummy_label_data = {
            "session_id": "test_session_123",
            "timestamp": time.time(),
            "description": "Dummy image for display test"
        }
        dummy_label = json.dumps(dummy_label_data)
        dummy_label = acquire_response.label
        print("Sending image and label for display...")
        display_request = acquisition_pb2.DisplayRequest(
            label=dummy_label,
            image=dummy_image_bytes
        )
        display_response = stub.display(display_request)
        print("Display request sent. Server should process and potentially display this.")
        print("Display method returned an empty response as expected.")

        # In a real application, you might have another mechanism to verify
        # if the display action on the server was successful (e.g., logging,
        # a callback, or checking a visual output).
        
    except grpc.RpcError as e:
        print(f"Display failed: {e}")

    channel.close()


In [4]:

# Input handler for Gradio: converts PIL image to bytes and puts into acquisition queue
def handle_input(image: Image.Image, session_id: str):
    """
    Handles input from Gradio UI. Converts PIL image to bytes and puts it
    into data_acq_queue along with a generated label.
    """
    if image is not None:
        try:
            # Create a simple JSON label for demonstration
            label_data = {
                "source": "gradio_input",
                "session_id": session_id if session_id else f"gradio_session_{int(time.time())}",
                "timestamp": time.time(),
                "image_format": "PNG"
            }
            label_json = json.dumps(label_data)
            
            image_bytes = image_to_bytes(image, format='PNG')
            data_acq_queue.put((label_json, image_bytes)) # Put tuple (label, image_bytes)
            logging.info(f"Handle INPUT: Put image ")
            
            # Immediately try to display something from the display queue if available
            return 
        except Exception as e:
            logging.exception(f"Gradio handle_input: Error processing image: {e}")
            return None, f"Error processing input: {e}"
    else:
        logging.info("Gradio handle_input: No image provided.")
        return # Still attempt to display existing data

# Output handler for Gradio: displays image from display queue
def display_img():
    """
    Retrieves image and info from data_display_queue for Gradio output.
    """
#    if not data_display_queue.empty():
    if True :
        try:
            logging.info(f"DISPLAY_IMG: GETTING FROM QUEUE - ")
            info, img_np = data_display_queue.get() # img_np is expected to be a numpy array from cv2.imdecode
            logging.info(f"DISPLAY_IMG: Displaying image (shape: {img_np.shape}) with info: {info}")
            # Gradio gr.Image(type="numpy") expects a numpy array
            yield img_np, info
        except queue.Empty:
            # Should not happen with empty check, but for safety
            logging.warning("Gradio display_img: Queue became empty before getting data.")
            yield None, "Waiting for image..."
        except Exception as e:
            logging.exception(f"Gradio display_img: Error getting data from display queue: {e}")
            yield None, f"Error displaying image: {e}"
    else:
        logging.info("Gradio display_img: Display queue is empty. Waiting for image...")
        yield None, "Waiting for image..."


In [ ]:
import gradio as gr

def acq_img():
    """Defines and returns the Gradio Blocks interface."""
    with gr.Blocks() as demo:
        gr.Markdown("# Image Acquisition & Display Service")
        gr.Markdown("Upload an image below to simulate acquisition. The gRPC server's `acquire` method can then fetch it.")
        gr.Markdown("Separately, you can trigger the gRPC `display` method from a client to send an image to this UI.")
        
        with gr.Row():
            with gr.Column():
                gr.Markdown("### Input to Acquisition Queue (for `acquire` method)")
                img_input = gr.Image(label="Upload Image (sends to acquisition queue)", type="pil")
                session_id_input = gr.Textbox(label="Optional Session ID", placeholder="e.g., my_session_1")
                # Add a button to explicitly trigger input handling, or rely on img_input.change
                # input_btn = gr.Button("Send to Acquisition Queue")

            with gr.Column():
                gr.Markdown("### Output from Display Queue (from `display` method)")
                img_output = gr.Image(label="Processed Image (from display queue)",streaming=True) # Expect numpy from cv2
                info_output = gr.Textbox(label="Processing Info (from display queue)")

        # Link input to handler
        # Using .change instead of .input for better reactivity if image is dropped/changed
        img_input.change(fn=handle_input, inputs=[img_input, session_id_input])
        #session_id_input.change(fn=handle_input, inputs=[img_input, session_id_input])
        img_output.stream(fn=display_img,outputs=[img_output, info_output],concurrency_limit=5,preprocess=False)
        img_output.change(fn=display_img,outputs=[img_output, info_output] )
        # To continuously poll for new images to display, use gr.Live
        # Note: gr.Live / gr.Blocks with .load and queue=True can be complex with gRPC.
        # For simplicity, we'll rely on the input trigger or manual refresh.
        # A more advanced setup might use websockets or server-sent events for continuous updates.
        
    return demo

demo=acq_img()

In [34]:
for i in range(1,10):
    run()
    print(i)

--- Testing Acquire Method ---
Attempting to acquire data...
Acquire successful!
Received label: {"source": "gradio_input", "session_id": "gradio_session_1753664053", "timestamp": 1753664053.418644, "image_format": "PNG"}
Acquired image saved to acquired_image_from_grpc.png

--- Testing Display Method ---
Sending image and label for display...
Display request sent. Server should process and potentially display this.
Display method returned an empty response as expected.
1
--- Testing Acquire Method ---
Attempting to acquire data...
Acquire successful!
Received label: {"source": "gradio_input", "session_id": "gradio_session_1753664080", "timestamp": 1753664080.3183875, "image_format": "PNG"}
Acquired image saved to acquired_image_from_grpc.png

--- Testing Display Method ---
Sending image and label for display...
Display request sent. Server should process and potentially display this.
Display method returned an empty response as expected.
2
--- Testing Acquire Method ---
Attempting to 

I0000 00:00:1753664124.799339    3083 chttp2_transport.cc:1287] ipv6:%5B::1%5D:8061: Got goaway [2] err=UNAVAILABLE:GOAWAY received; Error code: 2; Debug Text: Cancelling all calls {http2_error:2, grpc_status:14}


UnboundLocalError: local variable 'acquire_response' referenced before assignment